In [134]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [135]:
root = '../working/offline/6/0'
# context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
# pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-4.pooling_mask-attention_mask.grad_acc-4'
# pmodel2_name = 'deberta-v3-small.flag-pairwise14-4-cat.num_negs-4.eval.neg2'
# context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-2'
context_model_name = 'deberta-v3-small.flag-context3.dynamic_context_len.eval'
# context_model_name = 'deberta-v3-small.flag-context2-aug.cls_loss_rate-0.1.eval'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-2.pooling_mask-attention_mask'
# pmodel2_name = 'deberta-v3-small.flag-pairwise14-4-cat.num_negs-4.evalneg2.2'
pmodel2_name = 'cl.eval.neg2'

In [ ]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1.eval'
p2t_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
p2t_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample.eval'
context_model = 'deberta-v3-small.flag-context4-2-d'
context_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'

In [136]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [137]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [138]:
xp2 = gezi.load(f'{root}/{pmodel2_name}/valid.pkl')

In [139]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')

In [140]:
xc['cls_pred_ori'].shape

(424943, 100)

In [141]:
xp2['cls_pred_ori'].shape

(424943, 3)

In [142]:
xp2

{'id': array(['0002115f48f982', '00062ab8487156', '00062ab8487156', ...,
        'fff4714a37cf49', 'fffc1b4f08e81c', 'fffc1b4f08e81c'], dtype='<U14'),
 'cell_id': array(['9ec225f0', '96c8449c', 'aa354742', ..., 'f8744de1', '143cccfb',
        'ffc27279'], dtype='<U8'),
 'cid': array(['0002115f48f982\t9ec225f0', '00062ab8487156\t96c8449c',
        '00062ab8487156\taa354742', ..., 'fff4714a37cf49\tf8744de1',
        'fffc1b4f08e81c\t143cccfb', 'fffc1b4f08e81c\tffc27279'],
       dtype='<U23'),
 'cell_type': array(['markdown', 'markdown', 'markdown', ..., 'markdown', 'markdown',
        'markdown'], dtype='<U8'),
 'rank': array([ 0, 11,  5, ..., 17, 29,  0]),
 'rel_rank': array([0.05555556, 0.95454545, 0.5       , ..., 0.80769231, 0.73076923,
        0.01282051]),
 'match_code': array([ 0., 10.,  5., ..., 10., 28.,  0.]),
 'n_code_cell': array([ 8, 10, 10, ..., 12, 38, 38]),
 'n_markdown_cell': array([1, 2, 2, ..., 9, 2, 2]),
 'n_cell': array([ 9, 12, 12, ..., 21, 40, 40]),
 'code_idxes':

In [143]:
ids = set(xc['id'])

In [144]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a]"
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc326a, aa354742, 004d0eee, b9344b3d, 51709b61, 979a5b9a, af2b0426, 96c8449c]"
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e458, 465e9ac3, a1d5d434, 7b0d69c6, 8167cfe4, 8c027685, 490cee65, 24cfe6eb, 0ae3fd94, f9c995d1, e91fa12a, df9d54a6, b368450e, 2863864b, 9a829640, e6d30107, 5d2477a0, 01890991, 4a630d33, e902c3ad, 1f3749ad, a3ad710d, 7778a045, 95f37e93, 1f6fc839, a6b63513, 7cce9349, 65865e00, f8658f91, e92f292f, 489802aa, 11087190, 468d72f8, b789509e, a286a29c, 6812ae4b, 21ed7c42, e23919ef, 38b9ef54, dbd197c8, 647abaa1, 7bbcc573, 432a5f22, 9d7f5be8, ef7e6cfe, 7d149..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb7f4, de59dec8, 56c58dd6, 06fd00d9, a38c5996, c40c8f99, 60c81822, 4349c1aa, 4d1a0078, 7d32145e, 69383ee9, a7586092, d0c350c9, 98461fee, 25999581, b607b4cf, cb6a19b3, 11c8223a, 35705444, e0ff7e21, 68d8bd12, 62f32737, 90ce24a6, e2273244, 578cc8c7, b423993a, df5d3538, a44eecd0, 4e1213c3, 012dd24e, cdd2244f, 07daa78a, ab425c32]"
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501363, 94d3d43a, 9ecece20, 808d31ab, 615cd02a, 6da92b81, 568fa01a, 6603b9b9, dff53cea, 42476427, 3d63b500, bd24750b, 6306ba97, 592fa3ca, 70598ae5, 21992a54, 05915708, c6f0a7fb, 7da89797, fe931e4a, 57b14df5, ca9d112f, 9d2b623b, e0ffad5f, caa2a0e4, b6cb47da, 9d30d44b, 351cb961, cc2c96eb, 3d393ce3, ab3ae8d0, d9ae76ec, 9fed798a, 1a39fcb9, f9249928, a2f89492, 1ff43b24, dde2e9dd, 93b8809e, 3b9b6120, 6ff8d8cc, 30a931e1, 5fdb303a, fb5c88e1, 9df82070, a9e1c..."


In [145]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9009354223654629

In [146]:
df_p = pd.DataFrame(xp)

In [147]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [148]:
df_p.columns

Index(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred',
       'match_rank', 'match_code', 'n_code_cell', 'probs', 'sims'],
      dtype='object')

In [149]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [150]:
df_train = pd.read_feather('../working/train.fea')

In [151]:
df_train = df_train[df_train.id.isin(ids)]

In [152]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [153]:
gezi.set_fold(df, 5, 'ancestor_id')

In [154]:
from numba import jit
@jit(nopython=True)
def feat_speedup(arr):
    r_max, r_min, r_sum = -1e5,1e5,0
    for x in arr:
        r_max = max(r_max, x)
        r_min = min(r_min, x)
        r_sum += x
    return r_max, r_min, r_sum, r_sum/len(arr)

np_lin = np.linspace(0,1,7)

@jit(nopython=True)
def sorted_quantile(array, q):
    n = len(array)
    index = (n - 1) * q
    left = int(index)
    fraction = index - left
    right = left
    right = right + int(fraction > 0)
    i, j = array[left], array[right]
    return i + (j - i) * fraction

def get_percentile(array):
    x = np.sort(array)
    n = len(x)-1
    return x[[int(n*t) for t in np_lin[1:-1]]]


In [156]:
df['n_words'] = df['source'].apply(lambda x: len(x.split()))
df['n_lines'] = df['source'].apply(lambda x: len(x.split('\n')))
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5, top6 = [], [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob, top6_prob = [], [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim, top6_sim = [], [], [], [], []
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
entropy_p, entropy_c = [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
  # if len(idxes) > 5:
  #   top6.append((idxes[5] + 0.5) / (n_code + 1))
  #   top6_prob.append(probs[idxes[5]])
  #   top6_sim.append(sims[idxes[5]])
  # else:
  #   top6.append(-1)
  #   top6_prob.append(-1)
  #   top6_sim.append(-1)
  # dic = {}
  # for i,ntile in enumerate([sorted_quantile(probs,i) for i in np_lin]):
  #   dic[f'p_trend{i}'] = ntile
  # for i,ntile in enumerate(get_percentile(probs)):
  #   dic[f'p_ntile{i}'] = ntile
    
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])

  # for i,ntile in enumerate([sorted_quantile(probs,i) for i in np_lin]):
  #   dic[f'c_trend{i}'] = ntile
  # for i,ntile in enumerate(get_percentile(probs)):
  #   dic[f'c_ntile{i}'] = ntile
  # dics.append(dic)
  # ctail.append((idxes[-1] + 0.5) / FLAGS.num_classes)
  # ctail_prob.append(probs[idxes[-1]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
# dic = gezi.list2batch(dics)

df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_p2'] = xp2['pred']
df['cls_pred_p2'] = xp2['cls_pred']
p2_max_prob = []
p2_top2 = []
p2_top2_prob = []

for i in tqdm(range(len(df)), desc='p2'):
  code_idxes = xp2['code_idxes'][i]
  n_code = df['n_code_cell'].values[i]
  probs = gezi.softmax(xp2['cls_pred_ori'][i])
  idxes = (-probs).argsort()
  if df.max_prob.values[i] < 0.9:
    p2_top2.append((code_idxes[idxes[1]] + 0.5) / (n_code + 1))
    p2_max_prob.append(probs[idxes[0]])
    p2_top2_prob.append(probs[idxes[1]])
  else:
    p2_top2.append(df.top2.values[i])
    p2_max_prob.append(df.max_prob.values[i])
    p2_top2_prob.append(df.top2_prob.values[i])
    
df['p2_max_prob'] = p2_max_prob
df['p2_top2'] = p2_top2
df['p2_top2_prob'] = p2_top2_prob

df['abs_pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['abs_pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['abs_pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['abs_pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['abs_pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['abs_pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['abs_pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['abs_pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['abs_pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['abs_pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['abs_pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['abs_pred_diff11'] = abs(df['pred_c'] - df['pred_p2'])
df['abs_pred_diff12'] = abs(df['reg_pred'] - df['pred_p2'])
df['abs_pred_diff13'] = abs(df['cls_pred_c'] - df['pred_p2'])
df['abs_pred_diff14'] = abs(df['reg_pred'] - df['pred_p2'])
df['abs_pred_diff15'] = abs(df['cls2_pred'] - df['pred_p2'])
df['abs_pred_diff16'] = abs(df['reg_pred'] - df['cls_pred_c'])
df['abs_pred_diff17'] = abs(df['reg_pred'] - df['cls2_pred'])
df['abs_pred_diff18'] = abs(df['pred_p'] - df['pred_p2'])

df['pred_diff0'] = (df['pred_c'] - df['pred_p'])
df['pred_diff1'] = (df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = (df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = (df['cls2_pred'] - df['pred_p'])
df['pred_diff4'] = (df['pred_c'] - df['top2'])
df['pred_diff5'] = (df['reg_pred'] - df['top2'])
df['pred_diff6'] = (df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = (df['cls2_pred'] - df['top2'])
df['pred_diff8'] = (df['pred_p'] - df['top2'])
df['pred_diff9'] = (df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = (df['pred_p'] - df['top3'])
df['pred_diff11'] = (df['pred_c'] - df['pred_p2'])
df['pred_diff12'] = (df['reg_pred'] - df['pred_p2'])
df['pred_diff13'] = (df['cls_pred_c'] - df['pred_p2'])
df['pred_diff14'] = (df['reg_pred'] - df['pred_p2'])
df['pred_diff15'] = (df['cls2_pred'] - df['pred_p2'])
df['pred_diff16'] = (df['reg_pred'] - df['cls_pred_c'])
df['pred_diff17'] = (df['reg_pred'] - df['cls2_pred'])
df['pred_diff18'] = (df['pred_p'] - df['pred_p2'])
# df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]
# reg_pred_probs = []
# for row in tqdm(df.itertuples(), total=len(df), desc='cls_pred_p_prob'):
#   probs = row.probs
#   n_code = row.n_code_cell
#   reg_pred = row.reg_pred
#   try:
#     reg_pred_probs.append(probs[int(min(reg_pred, 1) * (n_code + 1) - 0.5)])
#   except Exception:
#     ic(n_code, len(probs), reg_pred)
#     break
# df['reg_pred_prob'] = reg_pred_probs
# for key in dic:
#   df[key] = dic[key]

top:   0%|          | 0/424943 [00:00<?, ?it/s]

p2:   0%|          | 0/424943 [00:00<?, ?it/s]

In [157]:
dfg = df.groupby('id')['reg_pred'].apply(list).reset_index(name='reg_preds')
dfg['reg_preds'] = dfg['reg_preds'].apply(lambda l: sorted(l))
dfg.head()

,id,reg_preds
0,0002115f48f982,[0.06091967970132828]
1,00062ab8487156,"[0.5682709813117981, 0.9557826519012451]"
2,000efd285fb982,"[0.010204765945672989, 0.024546556174755096, 0.03849152475595474, 0.09449729323387146, 0.11579092592000961, 0.1736208200454712, 0.19074419140815735, 0.195066899061203, 0.22131869196891785, 0.2851976454257965, 0.315576434135437, 0.31875061988830566, 0.337472528219223, 0.4146268367767334, 0.4352802634239197, 0.4583091735839844, 0.5087194442749023, 0.5711613297462463, 0.6207074522972107, 0.669959306716919, 0.701540470123291, 0.7497487664222717, 0.8268194198608398, 0.8876065611839294, 0.95594131..."
3,0012865b766949,"[0.013097383081912994, 0.013979047536849976, 0.03635818883776665, 0.06437728554010391, 0.15242360532283783, 0.17798539996147156, 0.2390294075012207, 0.3169291019439697, 0.3496144115924835, 0.37494218349456787, 0.39806729555130005, 0.40908488631248474, 0.4209931790828705, 0.44091126322746277, 0.49131160974502563, 0.6750174164772034, 0.6851706504821777, 0.7072246670722961, 0.8321345448493958, 0.8819966316223145, 0.9537708759307861, 0.9706590175628662, 0.9812918901443481]"
4,001308991e0c5e,"[0.015521883964538574, 0.03523843362927437, 0.0404181033372879, 0.15207649767398834, 0.1549975872039795, 0.18098345398902893, 0.229749858379364, 0.2329278290271759, 0.2385556399822235, 0.2933162748813629, 0.31128543615341187, 0.32982534170150757, 0.3495526611804962, 0.3570922613143921, 0.38254332542419434, 0.3839919865131378, 0.38969337940216064, 0.444924533367157, 0.4471125304698944, 0.45779120922088623, 0.5267796516418457, 0.5307557582855225, 0.5867294669151306, 0.6026049852371216, 0.63387..."


In [158]:
df = df.merge(dfg, on='id')

In [159]:
df['reg_pred_rank'] = [row.reg_preds.index(row.reg_pred) for row in tqdm(df.itertuples(), total=len(df))]
df['reg_pred_ratio'] = df['reg_pred_rank'] / df['n_markdown_cell']

  0%|          | 0/424943 [00:00<?, ?it/s]

In [160]:
# for i in range(len(np_lin)):
#   df[f'reg_pred_trend{i}'] = [sorted_quantile(row.reg_preds, np_lin[i]) for row in tqdm(df.itertuples(), total=len(df))]
# for i in range(len(np_lin) - 2):
#   df[f'reg_pred_ntile{i}'] = [get_percentile(row.reg_preds)[i] for row in tqdm(df.itertuples(), total=len(df))]

In [171]:
fold = 1
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [172]:
df.columns

Index(['id', 'cell_id', 'cid', 'cell_type', 'rank', 'rel_rank',
       'context_match', 'match_code', 'n_code_cell', 'n_markdown_cell',
       'n_cell', 'cls_pred_c', 'pred_c', 'reg_pred', 'cls_pred_ori',
       'match_rank', 'cls2_pred', 'pred_p', 'cls_pred_p', 'max_prob',
       'max_sim', 'probs', 'sims', 'ancestor_id', 'n_words', 'source', 'fold',
       'n_lines', 'markdown_frac', 'span', 'top2', 'top2_prob', 'top2_sim',
       'top3', 'top3_prob', 'top3_sim', 'top4', 'top4_prob', 'top4_sim',
       'top5', 'top5_prob', 'top5_sim', 'ctop_prob', 'ctop2', 'ctop2_prob',
       'ctop3', 'ctop3_prob', 'ctop4', 'ctop4_prob', 'pred_p2', 'cls_pred_p2',
       'p2_max_prob', 'p2_top2', 'p2_top2_prob', 'abs_pred_diff0',
       'abs_pred_diff1', 'abs_pred_diff2', 'abs_pred_diff3', 'abs_pred_diff4',
       'abs_pred_diff5', 'abs_pred_diff6', 'abs_pred_diff7', 'abs_pred_diff8',
       'abs_pred_diff9', 'abs_pred_diff10', 'abs_pred_diff11',
       'abs_pred_diff12', 'abs_pred_diff13', 'abs_pred

In [173]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'n_words',
          'n_lines',
          # 'cls_pred_c',
          # 'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          # 'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'pred_diff11',
          'pred_diff12',
          'pred_diff13',
          'pred_diff14',
          'pred_diff15',
          'pred_diff16',
          'pred_diff17',
          'pred_diff18',
          'abs_pred_diff0',
          'abs_pred_diff1',
          'abs_pred_diff2',
          'abs_pred_diff3',
          'abs_pred_diff4',
          'abs_pred_diff5',
          'abs_pred_diff6',
          'abs_pred_diff7',
          'abs_pred_diff8',
          'abs_pred_diff9',
          'abs_pred_diff10',
          'abs_pred_diff11',
          'abs_pred_diff12',
          'abs_pred_diff13',
          'abs_pred_diff14',
          'abs_pred_diff15',
          'abs_pred_diff16',
          'abs_pred_diff17',
          'abs_pred_diff18',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop3', 
          'ctop4', 
          'ctop2_prob', 
          'ctop3_prob', 
          'ctop4_prob',
          'pred_p2',
          'cls_pred_p2',
          'p2_max_prob',
          'p2_top2',
          'p2_top2_prob',
          'reg_pred_rank',
          'reg_pred_ratio',
          # 'entropy_p',
          # 'entropy_c',
          # 'entropy_p2',
          # 'p2_top3',
          # 'p2_top3_prob',
          # 'mean_cls_pred_p2',
        ]
# for i in range(FLAGS.num_classes):
#   reg_cols += [f'cls_pred{i}']
# for key in dic:
#   reg_cols += [key]
# for i in range(len(np_lin)):
#   reg_cols += [f'reg_pred_trend{i}'] 
# for i in range(len(np_lin) - 2):
#   reg_cols += [f'reg_pred_ntile{i}']
cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[label_col]
X_valid = dvalid[cols]
y_valid = dvalid[label_col]

In [174]:
from catboost import CatBoostRegressor, CatBoostRanker
cbt_params = {
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 100,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 300,
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

In [175]:
# model = CatBoostRanker(boosting_type="Plain", 
#                        loss_function="MAE", 
#                        eval_metric="MAE",
#                       #  n_estimators=10000, 
#                        n_estimators=1000,
#                        bootstrap_type="Bayesian", 
#                       #  early_stopping_rounds=100, 
#                        learning_rate=0.03, 
#                        max_depth=10,
#                        reg_lambda=7.960622217848342e-07,
#                       #  subsample=0.7422597612762745,
#                        verbose=False, random_state=1024)

In [176]:
# model.fit(X_train, y_train, group_id=dtrain.id.values,
#           eval_set=[(X_train, y_train), (X_valid, y_valid)],
#                 verbose=100,
#                 )  
# dvalid['cb_pred'] = model.predict(dvalid[cols])
# ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

In [177]:
model = CatBoostRegressor(**cbt_params)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=100,
                )  
dvalid['cb_pred'] = model.predict(dvalid[cols])
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

0:	learn: 0.2523171	test: 0.2523171	test1: 0.2522335	best: 0.2522335 (0)	total: 71.3ms	remaining: 11m 53s
100:	learn: 0.0689692	test: 0.0689692	test1: 0.0695718	best: 0.0695718 (100)	total: 5.25s	remaining: 8m 34s
200:	learn: 0.0624008	test: 0.0624008	test1: 0.0633931	best: 0.0633931 (200)	total: 10.2s	remaining: 8m 19s
300:	learn: 0.0611112	test: 0.0611112	test1: 0.0626024	best: 0.0626024 (300)	total: 15.3s	remaining: 8m 12s
400:	learn: 0.0600936	test: 0.0600936	test1: 0.0621281	best: 0.0621281 (400)	total: 20.2s	remaining: 8m 4s
500:	learn: 0.0592576	test: 0.0592576	test1: 0.0617656	best: 0.0617656 (500)	total: 25.2s	remaining: 7m 57s
600:	learn: 0.0585840	test: 0.0585840	test1: 0.0615208	best: 0.0615208 (600)	total: 30.3s	remaining: 7m 53s
700:	learn: 0.0579235	test: 0.0579235	test1: 0.0613118	best: 0.0613118 (700)	total: 35.2s	remaining: 7m 47s
800:	learn: 0.0573429	test: 0.0573429	test1: 0.0611916	best: 0.0611916 (800)	total: 40.3s	remaining: 7m 43s
900:	learn: 0.0568565	test: 0.0

[07/27/22 10:49:14] 2128221950.py:7 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9130395305862319


0.9130395305862319

In [178]:
gezi.plot.feature_importance(model, topn=20)

In [179]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          # 'num_leaves': 16,
          'min_data_in_leaf': 300,
          # 'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

bst = lgb.train(params,
                d_train,
                10000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=100)
dvalid['lgb_pred'] = bst.predict(dvalid[cols])
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030435 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18536
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 77
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.0715681	valid_1's l1: 0.0715125
[200]	training's l1: 0.0641803	valid_1's l1: 0.0641277
[300]	training's l1: 0.0628961	valid_1's l1: 0.0629513
[400]	training's l1: 0.0619634	valid_1's l1: 0.0621383
[500]	training's l1: 0.0613717	valid_1's l1: 0.0616531
[600]	training's l1: 0.0609336	valid_1's l1: 0.0613384
[700]	training's l1: 0.060634	valid_1's l1: 0.0611357
[800]	training's l1: 0.0604007	valid_1's l1: 0.0609989
[900]	training's l1: 0.0602019	valid_1's l1: 0.0608834
[1000]	training's l1: 0.0600224	valid_1's l1: 0.0607862
[1100]	training's l1: 0.0598739	valid_1's

0.9143370761557895

In [180]:
gezi.plot.feature_importance(bst, topn=20)

In [181]:
# gezi.plot.feature_importance_df(model)

In [182]:
# gezi.plot.feature_importance_df(bst)

In [183]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

0.9143370761557895

In [184]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})

0.9130395305862319

In [185]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values + dvalid.lgb_pred.values) / 2.})

0.9142124948887368

In [186]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})

0.9143961994689671

In [187]:
stop

NameError: name 'stop' is not defined

In [ ]:
len(df)

424943

In [ ]:
len(df[df.pred_p == df.pred_p2])

226847

In [ ]:
# df[df.pred_p==df.pred_p2][['pred_p', 'pred_p2']]

In [ ]:
df['x'] = ((df.pred_p * 1000).astype(int)==(df.pred_p2 * 1000).astype(int)).astype(int)

In [ ]:
gezi.set_pandas(precision=6)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df.pred_p.values[2]

0.49999999999999994

In [ ]:
df.pred_p2.values[2]

0.5

In [ ]:
df[['pred_p', 'pred_p2', 'max_prob', 'top2_prob', 'top3_prob', 'n_code_cell', 'x']].head(500)

,pred_p,pred_p2,max_prob,top2_prob,top3_prob,n_code_cell,x
0,0.055556,0.055556,0.942500,0.043493,0.006284,8,1
1,0.954545,0.954545,0.922126,0.064008,0.008434,10,1
2,0.500000,0.500000,0.769314,0.119769,0.050364,10,0
3,0.881579,0.881579,0.998220,0.001436,0.000290,37,1
4,0.328947,0.328947,0.981002,0.014497,0.003079,37,1
5,0.171053,0.171053,0.984083,0.006667,0.003236,37,1
6,0.118421,0.118421,0.777693,0.216999,0.003290,37,1
7,0.986842,0.986842,0.747022,0.206073,0.011413,37,1
8,0.618421,0.618421,0.484318,0.350390,0.093895,37,1
9,0.697368,0.697368,0.936894,0.031432,0.023601,37,1


In [ ]:
df.max_prob.describe(PERCENTILES2)

count   424943.000000
mean         0.724311
std          0.251772
min          0.035429
25%          0.517073
50%          0.786963
75%          0.966709
90%          0.995824
95%          0.998818
99%          0.999897
99.9%        0.999996
max          1.000000
Name: max_prob, dtype: float64

In [ ]:
len(df[df.max_prob < 0.9]) / len(df)

0.6237895435387805

In [ ]:
0.6237895435387805 * 3

1.8713686306163413

In [ ]:
len(df[df.max_prob < 0.8]) / len(df) * 3

1.5375403289382341

In [ ]:
0.5 * 3 + 0.2 * 2 

1.9

In [ ]:
len(df[df.max_prob > 0.95]) / len(df)

0.2920627001739057

In [ ]:
len(df[df.max_prob < 0.95]) / len(df)

0.7079372998260943

In [ ]:
d = df[(df.max_prob + df.top2_prob) < 0.9]
d[['pred_p', 'pred_p2', 'max_prob', 'top2_prob', 'top3_prob', 'n_code_cell', 'x']]

,pred_p,pred_p2,max_prob,top2_prob,top3_prob,n_code_cell,x
2,0.500000,0.500000,0.769314,0.119769,0.050364,10,0
8,0.618421,0.618421,0.484318,0.350390,0.093895,37,1
12,0.171053,0.092105,0.586005,0.246881,0.040511,37,0
14,0.276316,0.276316,0.386876,0.191654,0.085487,37,1
17,0.434211,0.407895,0.770194,0.079393,0.058807,37,0
...,...,...,...,...,...,...,...
424932,0.115385,0.192308,0.597850,0.193539,0.109157,12,0
424933,0.115385,0.115385,0.317720,0.192960,0.164996,12,1
424936,0.730769,0.730769,0.814843,0.071037,0.034016,12,1
424941,0.602564,0.730769,0.265239,0.252879,0.217327,38,0


In [ ]:
len(df[df.max_prob > 0.9]) / len(df)

0.3762104564612195

In [ ]:
len(df[(df.max_prob + df.top2_prob) > 0.8]) / len(df)

0.716456089404932

In [ ]:
0.37 * 0 + 0.34 * 2 + 0.27 * 3 + 0.03 * 4

1.6100000000000003

In [ ]:
len(df[(df.max_prob + df.top2_prob + df.top3_prob) > 0.5]) / len(df)

0.9720386028243788

In [ ]:
d = df[(df.max_prob + df.top2_prob + df.top3_prob) < 0.9]
d[['pred_p', 'pred_p2', 'max_prob', 'top2_prob', 'top3_prob', 'n_code_cell', 'rel_rank', 'reg_pred', 'top2', 'top3', 'x']].head(500)

,pred_p,pred_p2,max_prob,top2_prob,top3_prob,n_code_cell,rel_rank,reg_pred,top2,top3,x
12,0.171053,0.092105,0.586005,0.246881,0.040511,37,0.171053,0.115791,0.092105,0.197368,0
14,0.276316,0.276316,0.386876,0.191654,0.085487,37,0.322368,0.318751,0.486842,0.092105,1
21,0.171053,0.171053,0.582441,0.107087,0.095594,37,0.164474,0.195067,0.276316,0.118421,1
26,0.434211,0.328947,0.460383,0.200427,0.109199,37,0.328947,0.315576,0.355263,0.328947,0
51,0.033333,0.433333,0.270037,0.174595,0.150379,14,0.833333,0.832135,0.433333,0.700000,0
54,0.236842,0.236842,0.709028,0.096871,0.043270,56,0.236842,0.238556,0.254386,0.026316,1
65,0.973684,0.973684,0.695489,0.059501,0.054552,56,0.973684,0.981838,0.868421,0.675439,1
66,0.482456,0.482456,0.518166,0.227628,0.052233,56,0.479532,0.530756,0.307018,0.026316,1
69,0.657895,0.657895,0.460557,0.336530,0.053407,56,0.587719,0.652798,0.833333,0.885965,1
85,0.885965,0.868421,0.380749,0.255999,0.231037,56,0.868421,0.867114,0.868421,0.833333,0
